Citation: https://www.kaggle.com/code/baghern/a-deep-dive-into-sklearn-pipelines \
But with a lot of modifications! e.g throwing out the performance

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv('./data/train.csv')

df.dropna(axis=0)
df.set_index('id', inplace = True)

df.head()

,text,author
id,,
id26305,"This process, however, afforded me no means of...",EAP
id17569,It never once occurred to me that the fumbling...,HPL
id11008,"In his left hand was a gold snuff box, from wh...",EAP
id27763,How lovely is spring As we looked from Windsor...,MWS
id12958,"Finding nothing else, not even gold, the Super...",HPL


In [2]:
import re
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english'))

#creating a function to encapsulate preprocessing, to mkae it easy to replicate on  submission data
def processing(df):
    #lowering and removing punctuation
    df['processed'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))
    
    #numerical feature engineering
    #total length of sentence
    df['length'] = df['processed'].apply(lambda x: len(x))
    #get number of words
    df['words'] = df['processed'].apply(lambda x: len(x.split(' ')))
    df['words_not_stopword'] = df['processed'].apply(lambda x: len([t for t in x.split(' ') if t not in stopWords]))
    #get the average word length
    df['avg_word_length'] = df['processed'].apply(lambda x: np.mean([len(t) for t in x.split(' ') if t not in stopWords]) if len([len(t) for t in x.split(' ') if t not in stopWords]) > 0 else 0)
    #get the average word length
    df['commas'] = df['text'].apply(lambda x: x.count(','))

    return(df)

df = processing(df)

df.head()

,text,author,processed,length,words,words_not_stopword,avg_word_length,commas
id,,,,,,,,
id26305,"This process, however, afforded me no means of...",EAP,this process however afforded me no means of a...,224,41,21,6.380952,4
id17569,It never once occurred to me that the fumbling...,HPL,it never once occurred to me that the fumbling...,70,14,6,6.166667,0
id11008,"In his left hand was a gold snuff box, from wh...",EAP,in his left hand was a gold snuff box from whi...,195,36,19,5.947368,4
id27763,How lovely is spring As we looked from Windsor...,MWS,how lovely is spring as we looked from windsor...,202,34,21,6.476190,3
id12958,"Finding nothing else, not even gold, the Super...",HPL,finding nothing else not even gold the superin...,170,27,16,7.187500,2


In [14]:
from sklearn.model_selection import train_test_split

features= [c for c in df.columns.values if c  not in ['id','text','author']]
numeric_features= [c for c in df.columns.values if c  not in ['id','text','author','processed']]
target = 'author'

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.1, random_state=42)
X_train.head()

,processed,length,words,words_not_stopword,avg_word_length,commas
id,,,,,,
id05994,still others including joe himself have theori...,90,14,9,6.333333,2
id03819,they struck a solid wooden substance which ext...,124,23,10,6.200000,1
id25304,again i rose and exerting all the firmness of ...,140,27,10,6.400000,2
id16857,why did he write them,21,5,1,5.000000,0
id06134,your curiosity makes you irresponsible,38,5,3,9.000000,0


In [15]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

text = Pipeline([
                ('selector', TextSelector(key='processed')),
                ('tfidf', TfidfVectorizer( stop_words='english'))
            ])

pipeline = Pipeline([
    ('text',text),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_test)
np.mean(preds == y_test)

0.7139938712972421

In [17]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'text', 'classifier', 'text__memory', 'text__steps', 'text__verbose', 'text__selector', 'text__tfidf', 'text__selector__key', 'text__tfidf__analyzer', 'text__tfidf__binary', 'text__tfidf__decode_error', 'text__tfidf__dtype', 'text__tfidf__encoding', 'text__tfidf__input', 'text__tfidf__lowercase', 'text__tfidf__max_df', 'text__tfidf__max_features', 'text__tfidf__min_df', 'text__tfidf__ngram_range', 'text__tfidf__norm', 'text__tfidf__preprocessor', 'text__tfidf__smooth_idf', 'text__tfidf__stop_words', 'text__tfidf__strip_accents', 'text__tfidf__sublinear_tf', 'text__tfidf__token_pattern', 'text__tfidf__tokenizer', 'text__tfidf__use_idf', 'text__tfidf__vocabulary', 'classifier__bootstrap', 'classifier__ccp_alpha', 'classifier__class_weight', 'classifier__criterion', 'classifier__max_depth', 'classifier__max_features', 'classifier__max_leaf_nodes', 'classifier__max_samples', 'classifier__min_impurity_decrease', 'classifier__min_samples_leaf', 'class

In [30]:
from sklearn.model_selection import GridSearchCV
import math 
import time

def my_scorer(y_true, y_predicted):
    # https://stackoverflow.com/questions/46208221/scikit-learn-gridsearch-custom-scoring-function
    error = np.mean(y_true == y_predicted)
    return error

from sklearn.metrics import make_scorer
scoring_func = make_scorer(my_scorer, greater_is_better=True)


hyperparameters = { 'text__tfidf__max_df': [0.9, 0.95],
                    'text__tfidf__ngram_range': [(1,1), (1,2)],
                    'classifier__min_samples_leaf': [1,2]
                  }
#                    'classifier__max_depth': [50, 70], # the max depth parameter needs to be used carefully or lowers performance

start_time = time.time()
clf = GridSearchCV(pipeline, hyperparameters, cv=5, scoring=scoring_func)
# Fit and tune model
clf.fit(X_train, y_train) 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('text',
                                        Pipeline(steps=[('selector',
                                                         TextSelector(key='processed')),
                                                        ('tfidf',
                                                         TfidfVectorizer(stop_words='english'))])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             param_grid={'classifier__min_samples_leaf': [1, 2],
                         'text__tfidf__max_df': [0.9, 0.95],
                         'text__tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(my_scorer))

In [31]:
print("Total Time:", time.time()-start_time)

Total Time: 1296.3024628162384


In [32]:
clf.best_params_

{'classifier__min_samples_leaf': 2,
 'text__tfidf__max_df': 0.9,
 'text__tfidf__ngram_range': (1, 2)}

In [33]:
#refitting on entire training data using best settings
clf.refit

preds = clf.predict(X_test)
probs = clf.predict_proba(X_test)


print(np.mean(preds == y_test))

y_test[:5]

0.7334014300306435


id
id15695    EAP
id07954    MWS
id16303    MWS
id07932    EAP
id20875    HPL
Name: author, dtype: object